In [2]:
%pip install sqlalchemy pyodbc matplotlib seaborn plotly

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import pyodbc
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Database connection configuration
# Replace with your actual credentials
server = 'upgrade-abnb-server.database.windows.net'
database = 'Upgrade_Abnb'
username = 'vmabnbserver'  # Replace with actual username
password = 'JWkWW8Bg%>jy,Xj!'  # Replace with actual password

# Function to test ODBC drivers and create connection
def create_db_connection():
    """Create database connection with fallback driver options"""
    drivers = [
        'ODBC Driver 18 for SQL Server',
        'ODBC Driver 17 for SQL Server',
        'ODBC Driver 13 for SQL Server',
        'SQL Server Native Client 11.0',
        'SQL Server'
    ]
    
    # Test available drivers
    available_drivers = [driver for driver in pyodbc.drivers() if any(d in driver for d in ['SQL Server', 'ODBC'])]
    print(f"Available ODBC drivers: {available_drivers}")
    
    for driver in drivers:
        if driver in available_drivers:
            try:
                connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}&TrustServerCertificate=yes&Encrypt=yes'
                engine = create_engine(connection_string)
                # Test connection
                with engine.connect() as conn:
                    conn.execute(sqlalchemy.text("SELECT 1"))
                print(f"Successfully connected using driver: {driver}")
                return engine, connection_string
            except Exception as e:
                print(f"Failed with driver {driver}: {str(e)}")
                continue
    
    # If all drivers fail, try without encryption
    for driver in drivers:
        if driver in available_drivers:
            try:
                connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}&TrustServerCertificate=yes'
                engine = create_engine(connection_string)
                # Test connection
                with engine.connect() as conn:
                    conn.execute(sqlalchemy.text("SELECT 1"))
                print(f"Successfully connected using driver (no encryption): {driver}")
                return engine, connection_string
            except Exception as e:
                print(f"Failed with driver {driver} (no encryption): {str(e)}")
                continue
    
    raise Exception("Could not establish database connection with any available driver")

# Create connection engine
engine, connection_string = create_db_connection()

# Function to execute queries and return DataFrames
def query_to_df(query, engine=engine):
    """Execute SQL query and return DataFrame"""
    return pd.read_sql(query, engine)

# Load main dataset
query = "SELECT * FROM [dbo].[listings_completo]"
df_listings = query_to_df(query)

# Configure matplotlib and seaborn for better visualizations
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Display basic information
print("=== DATABASE CONNECTION ESTABLISHED ===")
print(f"Connected to: {database}")
print(f"DataFrame shape: {df_listings.shape}")
print(f"Columns: {list(df_listings.columns)}")
print("\n=== FIRST 5 ROWS ===")
print(df_listings.head())
print("\n=== DATA TYPES ===")
print(df_listings.dtypes)
print("\n=== MISSING VALUES ===")
print(df_listings.isnull().sum())

Available ODBC drivers: ['SQL Server']
Successfully connected using driver: SQL Server
=== DATABASE CONNECTION ESTABLISHED ===
Connected to: Upgrade_Abnb
DataFrame shape: (1630091, 63)
Columns: ['listing_id', 'id_x', 'date', 'reviewer_id', 'reviewer_name', 'id_y', 'name', 'host_id', 'host_name', 'host_since', 'host_location', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'price', 'minimum_nights', 'maximum_nights', 'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'calendar_last_scraped', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'availability_eoy', 'number_of_reviews_ly', 'estimated_occupancy_l365d', 'estimated_revenue_l365d

In [11]:
import folium
from folium import plugins
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

# Crear un mapa interactivo mostrando ciudades con métricas clave
def create_interactive_map(city_stats):
    # Calcular el centro del mapa basado en coordenadas promedio
    center_lat = city_stats['latitude'].mean()
    center_lon = city_stats['longitude'].mean()
    
    # Crear mapa base
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=6,
        tiles='CartoDB positron'
    )
    
    # Añadir marcadores para cada ciudad
    for idx, row in city_stats.iterrows():
        # Crear contenido popup con métricas clave
        popup_content = f"""
        <div style="font-family: Arial; width: 250px;">
            <h4 style="color: #FF6B6B; margin-bottom: 10px;">🏠 {row['origen']}</h4>
            <hr style="margin: 5px 0;">
            <p><b>💰 Precio Promedio:</b> €{row['avg_price']:.0f}</p>
            <p><b>🏘️ Total Anuncios:</b> {row['total_listings']:,}</p>
            <p><b>⭐ Satisfacción:</b> {row['avg_satisfaction']:.2f}/5.0</p>
            <p><b>📈 Ocupación:</b> {row['avg_occupancy']:.0f}%</p>
            <p><b>💸 Ingresos Promedio:</b> €{row['avg_revenue']:.0f}</p>
        </div>
        """
        
        # Codificación de colores basada en nivel de precio
        if row['avg_price'] > 300:
            color = '#FF6B6B'  # Rojo para caro
        elif row['avg_price'] > 150:
            color = '#45B7D1'  # Azul para medio
        else:
            color = '#4ECDC4'  # Verde azulado para asequible
        
        # Añadir marcador circular con tamaño basado en número de anuncios
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=max(10, row['total_listings'] / 500),  # Escalar tamaño del marcador
            popup=folium.Popup(popup_content, max_width=300),
            color='white',
            weight=3,
            fillColor=color,
            fillOpacity=0.8,
            tooltip=f"{row['origen']} - €{row['avg_price']:.0f}"
        ).add_to(m)
    
    # Añadir leyenda
    legend_html = '''
    <div style="position: fixed; 
                top: 10px; right: 10px; width: 200px; height: 120px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:14px; font-family: Arial; padding: 10px;">
    <h4 style="margin-top:0; color: #333;">Niveles de Precio</h4>
    <p><span style="color:#FF6B6B;">●</span> Premium (>€300)</p>
    <p><span style="color:#45B7D1;">●</span> Medio (€150-300)</p>
    <p><span style="color:#4ECDC4;">●</span> Asequible (<€150)</p>
    <p><i>Tamaño marcador = # anuncios</i></p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    return m

# Crear y mostrar el mapa
map_viz = create_interactive_map(city_stats)
map_viz

# Crear un dashboard moderno integral con mejor organización
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=[
        '💰 Análisis de Precios', '🏠 Tamaño del Mercado', '⭐ Métricas de Calidad',
        '📊 Ocupación vs Ingresos', '🎯 Matriz de Rendimiento', '📈 Distribución de Ingresos',
        '🌍 Vista Geográfica', '📋 Resumen del Mercado', '🏆 Mejores Rendimientos'
    ],
    specs=[
        [{"type": "bar"}, {"type": "pie"}, {"type": "bar"}],
        [{"type": "scatter"}, {"type": "scatter"}, {"type": "histogram"}],
        [{"type": "bar"}, {"type": "table"}, {"type": "bar"}]
    ],
    vertical_spacing=0.08,
    horizontal_spacing=0.05
)

# Esquema de colores
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

# 1. Análisis de Precios (Fila 1, Col 1)
fig.add_trace(
    go.Bar(
        x=city_stats['origen'],
        y=city_stats['avg_price'],
        name='Precio Promedio',
        marker_color=colors,
        text=[f'€{price:.0f}' for price in city_stats['avg_price']],
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Precio: €%{y:.0f}<extra></extra>'
    ),
    row=1, col=1
)

# 2. Gráfico de Pastel Tamaño del Mercado (Fila 1, Col 2)
fig.add_trace(
    go.Pie(
        labels=city_stats['origen'],
        values=city_stats['total_listings'],
        name="Cuota de Mercado",
        marker_colors=colors,
        textinfo='label+percent',
        hovertemplate='<b>%{label}</b><br>Anuncios: %{value:,}<br>Cuota: %{percent}<extra></extra>'
    ),
    row=1, col=2
)

# 3. Métricas de Calidad (Fila 1, Col 3)
fig.add_trace(
    go.Bar(
        x=city_stats['origen'],
        y=city_stats['avg_satisfaction'],
        name='Satisfacción',
        marker_color=colors,
        text=[f'{rating:.2f}' for rating in city_stats['avg_satisfaction']],
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Puntuación: %{y:.2f}/5<extra></extra>'
    ),
    row=1, col=3
)

# 4. Ocupación vs Ingresos (Fila 2, Col 1)
fig.add_trace(
    go.Scatter(
        x=city_stats['avg_occupancy'],
        y=city_stats['avg_revenue'],
        mode='markers+text',
        marker=dict(
            size=city_stats['total_listings']/10000,  # Reduced divisor for better visibility
            color=colors,
            line=dict(width=2, color='white')
        ),
        text=city_stats['origen'],
        textposition='top center',
        name='Rendimiento',
        hovertemplate='<b>%{text}</b><br>Ocupación: %{x:.0f}%<br>Ingresos: €%{y:.0f}<extra></extra>'
    ),
    row=2, col=1
)

# 5. Matriz de Rendimiento - Precio vs Satisfacción (Fila 2, Col 2)
fig.add_trace(
    go.Scatter(
        x=city_stats['avg_price'],
        y=city_stats['avg_satisfaction'],
        mode='markers+text',
        marker=dict(
            size=city_stats['total_listings']/10000,  # Reduced divisor for better visibility
            color=colors,
            line=dict(width=2, color='white')
        ),
        text=city_stats['origen'],
        textposition='top center',
        name='Precio vs Calidad',
        hovertemplate='<b>%{text}</b><br>Precio: €%{x:.0f}<br>Puntuación: %{y:.2f}/5<extra></extra>'
    ),
    row=2, col=2
)

# 6. Distribución de Ingresos (Fila 2, Col 3)
fig.add_trace(
    go.Bar(
        x=city_stats['origen'],
        y=city_stats['avg_revenue'],
        name='Ingresos',
        marker_color=colors,
        text=[f'€{rev:.0f}' for rev in city_stats['avg_revenue']],
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Ingresos: €%{y:.0f}<extra></extra>'
    ),
    row=2, col=3
)

# 7. Distribución Geográfica de Precios (Fila 3, Col 1)
fig.add_trace(
    go.Bar(
        x=['Norte España', 'Este España', 'Sur España'],
        y=[city_stats.iloc[2]['avg_price'], city_stats.iloc[2]['avg_price'], city_stats.iloc[1]['avg_price']],
        name='Precios Regionales',
        marker_color=['#96CEB4', '#FFEAA7', '#DDA0DD'],
        hovertemplate='<b>%{x}</b><br>Precio Promedio: €%{y:.0f}<extra></extra>'
    ),
    row=3, col=1
)

# 8. Tabla Resumen del Mercado (Fila 3, Col 2)
summary_data = [
    ['Total Mercados', f'{len(city_stats)}'],
    ['Total Anuncios', f'{city_stats["total_listings"].sum():,}'],
    ['Precio Promedio', f'€{city_stats["avg_price"].mean():.0f}'],
    ['Puntuación Promedio', f'{city_stats["avg_satisfaction"].mean():.2f}/5'],
    ['Ocupación Promedio', f'{city_stats["avg_occupancy"].mean():.0f}%']
]

fig.add_trace(
    go.Table(
        header=dict(values=['Métrica', 'Valor'],
                   fill_color='#FF6B6B',
                   font=dict(color='white', size=12)),
        cells=dict(values=[[row[0] for row in summary_data],
                          [row[1] for row in summary_data]],
                  fill_color='#f8f9fa',
                  font=dict(size=11))
    ),
    row=3, col=2
)

# 9. Mejores Rendimientos (Fila 3, Col 3)
performance_score = (
    (city_stats['avg_satisfaction'] / 5) * 0.3 +
    (city_stats['avg_occupancy'] / city_stats['avg_occupancy'].max()) * 0.3 +
    (city_stats['avg_revenue'] / city_stats['avg_revenue'].max()) * 0.4
) * 100

fig.add_trace(
    go.Bar(
        x=city_stats['origen'],
        y=performance_score,
        name='Puntuación de Rendimiento',
        marker_color=colors,
        text=[f'{score:.1f}%' for score in performance_score],
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Puntuación: %{y:.1f}%<extra></extra>'
    ),
    row=3, col=3
)

# Actualizar diseño
fig.update_layout(
    title=dict(
        text="🏠 Dashboard Integral de Análisis del Mercado Airbnb",
        x=0.5,
        font=dict(size=24, family="Arial Black")
    ),
    height=1200,
    showlegend=False,
    template="plotly_white",
    font=dict(family="Arial", size=10)
)

# Actualizar ejes específicos
fig.update_xaxes(title_text="Ciudad", row=1, col=1)
fig.update_yaxes(title_text="Precio (€)", row=1, col=1)
fig.update_yaxes(title_text="Puntuación (1-5)", row=1, col=3)
fig.update_xaxes(title_text="Ocupación (%)", row=2, col=1)
fig.update_yaxes(title_text="Ingresos (€)", row=2, col=1)
fig.update_xaxes(title_text="Precio (€)", row=2, col=2)
fig.update_yaxes(title_text="Satisfacción", row=2, col=2)
fig.update_xaxes(title_text="Ciudad", row=2, col=3)
fig.update_yaxes(title_text="Ingresos (€)", row=2, col=3)
fig.update_xaxes(title_text="Región", row=3, col=1)
fig.update_yaxes(title_text="Precio (€)", row=3, col=1)
fig.update_xaxes(title_text="Ciudad", row=3, col=3)
fig.update_yaxes(title_text="Puntuación (%)", row=3, col=3)

fig.show()

# Resumen ejecutivo mejorado con perspectivas geográficas
print("=" * 100)
print("🌍 ANÁLISIS INTEGRAL DEL MERCADO AIRBNB - PERSPECTIVAS GEOGRÁFICAS")
print("=" * 100)
print(f"📊 Mercados Analizados: {len(city_stats)} destinos principales españoles")
print(f"🏠 Total Propiedades: {city_stats['total_listings'].sum():,} anuncios activos")
print(f"💰 Precio Promedio del Mercado: €{city_stats['avg_price'].mean():.0f} por noche")
print(f"⭐ Satisfacción General de Huéspedes: {city_stats['avg_satisfaction'].mean():.2f}/5.0")
print(f"📈 Ocupación Promedio del Mercado: {city_stats['avg_occupancy'].mean():.0f}%")
print(f"💸 Ingresos Totales del Mercado: €{city_stats['avg_revenue'].sum():,.0f}")
print("=" * 100)

print("🗺️ DISTRIBUCIÓN GEOGRÁFICA:")
for idx, row in city_stats.iterrows():
    region = "Isla Mediterránea" if row['origen'] == "Mallorca" else "Costa Mediterránea"
    print(f"   📍 {row['origen']} ({region})")
    print(f"      🎯 Posición en el Mercado: {row['total_listings']:,} anuncios | €{row['avg_price']:.0f}/noche")
    print(f"      📊 Rendimiento: {row['avg_satisfaction']:.2f}★ | {row['avg_occupancy']:.0f}% ocupación")
    print()

print("🏆 LÍDERES DEL MERCADO POR CATEGORÍA:")
print(f"💎 Líder Mercado Premium: {city_stats.loc[city_stats['avg_price'].idxmax(), 'origen']} (€{city_stats['avg_price'].max():.0f}/noche)")
print(f"⭐ Líder Satisfacción Huéspedes: {city_stats.loc[city_stats['avg_satisfaction'].idxmax(), 'origen']} ({city_stats['avg_satisfaction'].max():.2f}/5.0)")
print(f"📊 Líder Tamaño del Mercado: {city_stats.loc[city_stats['total_listings'].idxmax(), 'origen']} ({city_stats['total_listings'].max():,} anuncios)")
print(f"📈 Líder Ocupación: {city_stats.loc[city_stats['avg_occupancy'].idxmax(), 'origen']} ({city_stats['avg_occupancy'].max():.0f}%)")
print(f"💸 Líder Ingresos: {city_stats.loc[city_stats['avg_revenue'].idxmax(), 'origen']} (€{city_stats['avg_revenue'].max():.0f})")
print("=" * 100)

# Perspectivas del mercado
price_range = city_stats['avg_price'].max() - city_stats['avg_price'].min()
print("📈 PERSPECTIVAS CLAVE DEL MERCADO:")
print(f"   💰 Variación de Precios: €{price_range:.0f} diferencia entre mercados")
print(f"   🎯 Concentración del Mercado: {(city_stats['total_listings'].max()/city_stats['total_listings'].sum()*100):.1f}% de anuncios en el mercado más grande")
print(f"   ⭐ Consistencia de Calidad: {city_stats['avg_satisfaction'].std():.2f} desviación estándar en satisfacción")
print(f"   📊 Eficiencia de Rendimiento: {city_stats['avg_occupancy'].mean():.0f}% tasa de utilización promedio")
print("=" * 100)

🌍 ANÁLISIS INTEGRAL DEL MERCADO AIRBNB - PERSPECTIVAS GEOGRÁFICAS
📊 Mercados Analizados: 3 destinos principales españoles
🏠 Total Propiedades: 1,630,091 anuncios activos
💰 Precio Promedio del Mercado: €219 por noche
⭐ Satisfacción General de Huéspedes: 4.75/5.0
📈 Ocupación Promedio del Mercado: 58%
💸 Ingresos Totales del Mercado: €135,692
🗺️ DISTRIBUCIÓN GEOGRÁFICA:
   📍 Mallorca (Isla Mediterránea)
      🎯 Posición en el Mercado: 415,807 anuncios | €396/noche
      📊 Rendimiento: 4.77★ | 55% ocupación

   📍 Málaga (Costa Mediterránea)
      🎯 Posición en el Mercado: 771,561 anuncios | €115/noche
      📊 Rendimiento: 4.74★ | 60% ocupación

   📍 Valencia (Costa Mediterránea)
      🎯 Posición en el Mercado: 442,723 anuncios | €145/noche
      📊 Rendimiento: 4.74★ | 58% ocupación

🏆 LÍDERES DEL MERCADO POR CATEGORÍA:
💎 Líder Mercado Premium: Mallorca (€396/noche)
⭐ Líder Satisfacción Huéspedes: Mallorca (4.77/5.0)
📊 Líder Tamaño del Mercado: Málaga (771,561 anuncios)
📈 Líder Ocupación: Má

In [7]:
# First, let's calculate city statistics if not already available
try:
    city_stats
except NameError:
    # Calculate city statistics from the listings data
    city_stats = df_listings.groupby('origen').agg({
        'price': 'mean',
        'id_y': 'count',
        'review_scores_rating': 'mean',
        'availability_365': lambda x: ((365 - x.mean()) / 365 * 100),
        'latitude': 'mean',
        'longitude': 'mean'
    }).round(2)
    
    city_stats.columns = ['avg_price', 'total_listings', 'avg_satisfaction', 'avg_occupancy', 'latitude', 'longitude']
    city_stats['avg_revenue'] = city_stats['avg_price'] * (city_stats['avg_occupancy'] / 100) * 365
    city_stats = city_stats.reset_index()
    # No need to rename since we're already grouping by 'origen'

# Get coordinates for major Spanish cities if not in data
city_coordinates = {
    'Barcelona': (41.3851, 2.1734),
    'Madrid': (40.4168, -3.7038),
    'Valencia': (39.4699, -0.3763),
    'Sevilla': (37.3891, -5.9845),
    'Málaga': (36.7213, -4.4214),
    'Mallorca': (39.6953, 2.9603),
    'Bilbao': (43.2627, -2.9253),
    'San Sebastián': (43.3183, -1.9812),
    'Granada': (37.1773, -3.5986),
    'Córdoba': (37.8882, -4.7794)
}

# Update coordinates if missing or incorrect
for idx, row in city_stats.iterrows():
    city = row['origen']
    if city in city_coordinates:
        city_stats.loc[idx, 'latitude'] = city_coordinates[city][0]
        city_stats.loc[idx, 'longitude'] = city_coordinates[city][1]

# Create interactive map with real Spanish geography
def create_real_spain_map(city_stats):
    # Center map on Spain
    spain_center = [40.0, -4.0]
    
    # Create map focused on Spain
    m = folium.Map(
        location=spain_center,
        zoom_start=6,
        tiles='OpenStreetMap'
    )
    
    # Add different tile layers for better visualization
    folium.TileLayer('CartoDB Positron').add_to(m)
    folium.TileLayer('CartoDB Dark_Matter').add_to(m)
    
    # Create color scale based on price levels
    max_price = city_stats['avg_price'].max()
    min_price = city_stats['avg_price'].min()
    
    def get_color(price):
        if price > max_price * 0.8:
            return '#FF4444'  # Red for expensive
        elif price > max_price * 0.6:
            return '#FF8C00'  # Orange for medium-high
        elif price > max_price * 0.4:
            return '#FFD700'  # Gold for medium
        else:
            return '#32CD32'  # Green for affordable
    
    # Add markers for each city
    for idx, row in city_stats.iterrows():
        # Create detailed popup with all metrics
        popup_html = f"""
        <div style="font-family: 'Segoe UI', Arial, sans-serif; width: 300px; padding: 15px;">
            <h3 style="color: #2C3E50; margin: 0 0 15px 0; text-align: center; border-bottom: 2px solid #3498DB;">
                🏛️ {row['origen']}
            </h3>
            
            <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                        color: white; padding: 10px; border-radius: 8px; margin-bottom: 15px;">
                <h4 style="margin: 0; text-align: center;">💰 €{row['avg_price']:.0f} por noche</h4>
            </div>
            
            <table style="width: 100%; border-collapse: collapse;">
                <tr style="background-color: #f8f9fa;">
                    <td style="padding: 8px; border: 1px solid #dee2e6;"><strong>🏠 Propiedades:</strong></td>
                    <td style="padding: 8px; border: 1px solid #dee2e6;">{row['total_listings']:,}</td>
                </tr>
                <tr>
                    <td style="padding: 8px; border: 1px solid #dee2e6;"><strong>⭐ Satisfacción:</strong></td>
                    <td style="padding: 8px; border: 1px solid #dee2e6;">{row['avg_satisfaction']:.2f}/5.0</td>
                </tr>
                <tr style="background-color: #f8f9fa;">
                    <td style="padding: 8px; border: 1px solid #dee2e6;"><strong>📈 Ocupación:</strong></td>
                    <td style="padding: 8px; border: 1px solid #dee2e6;">{row['avg_occupancy']:.0f}%</td>
                </tr>
                <tr>
                    <td style="padding: 8px; border: 1px solid #dee2e6;"><strong>💸 Ingresos Anuales:</strong></td>
                    <td style="padding: 8px; border: 1px solid #dee2e6;">€{row['avg_revenue']:.0f}</td>
                </tr>
                <tr style="background-color: #f8f9fa;">
                    <td style="padding: 8px; border: 1px solid #dee2e6;"><strong>📍 Coordenadas:</strong></td>
                    <td style="padding: 8px; border: 1px solid #dee2e6;">{row['latitude']:.3f}, {row['longitude']:.3f}</td>
                </tr>
            </table>
            
            <div style="margin-top: 15px; text-align: center; font-size: 12px; color: #7f8c8d;">
                💡 Tamaño del marcador = Volumen de mercado
            </div>
        </div>
        """
        
        # Calculate marker size based on market volume
        marker_size = max(15, min(40, row['total_listings'] / 100))
        
        # Add circle marker
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=marker_size,
            popup=folium.Popup(popup_html, max_width=350),
            tooltip=f"{row['origen']} - €{row['avg_price']:.0f}/noche - {row['total_listings']} propiedades",
            color='white',
            weight=3,
            fillColor=get_color(row['avg_price']),
            fillOpacity=0.8
        ).add_to(m)
        
        # Add city label
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            icon=folium.DivIcon(
                html=f"""
                <div style="background-color: rgba(255,255,255,0.9); 
                           border: 2px solid {get_color(row['avg_price'])}; 
                           border-radius: 5px; 
                           padding: 2px 6px; 
                           font-weight: bold; 
                           font-size: 12px; 
                           color: #2C3E50;
                           box-shadow: 0 2px 4px rgba(0,0,0,0.3);">
                    {row['origen']}
                </div>
                """,
                icon_size=(80, 20),
                icon_anchor=(40, 35)
            )
        ).add_to(m)
    
    # Add comprehensive legend
    legend_html = f"""
    <div style="position: fixed; 
                top: 10px; right: 10px; width: 280px; 
                background-color: white; border: 3px solid #2C3E50; z-index: 9999; 
                font-size: 14px; font-family: 'Segoe UI', Arial; 
                padding: 15px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.3);">
        
        <h3 style="margin: 0 0 15px 0; color: #2C3E50; text-align: center; border-bottom: 2px solid #3498DB; padding-bottom: 5px;">
            🗺️ Mapa del Mercado Airbnb España
        </h3>
        
        <h4 style="margin: 10px 0 5px 0; color: #34495E;">💰 Niveles de Precio:</h4>
        <p style="margin: 3px 0;"><span style="color: #FF4444; font-size: 18px;">●</span> Premium: >€{max_price * 0.8:.0f}</p>
        <p style="margin: 3px 0;"><span style="color: #FF8C00; font-size: 18px;">●</span> Alto: €{max_price * 0.6:.0f}-{max_price * 0.8:.0f}</p>
        <p style="margin: 3px 0;"><span style="color: #FFD700; font-size: 18px;">●</span> Medio: €{max_price * 0.4:.0f}-{max_price * 0.6:.0f}</p>
        <p style="margin: 3px 0;"><span style="color: #32CD32; font-size: 18px;">●</span> Asequible: <€{max_price * 0.4:.0f}</p>
        
        <hr style="margin: 10px 0; border: 1px solid #BDC3C7;">
        
        <h4 style="margin: 10px 0 5px 0; color: #34495E;">📊 Información:</h4>
        <p style="margin: 3px 0; font-size: 12px;">🔵 Tamaño = Volumen de mercado</p>
        <p style="margin: 3px 0; font-size: 12px;">📍 Click para detalles completos</p>
        <p style="margin: 3px 0; font-size: 12px;">🗺️ Múltiples capas disponibles</p>
        
        <div style="margin-top: 10px; padding: 8px; background-color: #ECF0F1; border-radius: 5px; text-align: center;">
            <strong style="color: #2C3E50;">Total: {len(city_stats)} ciudades</strong><br>
            <span style="font-size: 12px; color: #7F8C8D;">{city_stats['total_listings'].sum():,} propiedades</span>
        </div>
    </div>
    """
    
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    # Add fullscreen button
    plugins.Fullscreen().add_to(m)
    
    # Add measure control
    plugins.MeasureControl().add_to(m)
    
    return m

# Import required libraries for the map
import folium
from folium import plugins

# Create and display the real Spain map
print("🗺️ Creando mapa interactivo de España con datos reales de Airbnb...")
spain_map = create_real_spain_map(city_stats)

# Display the map
spain_map

🗺️ Creando mapa interactivo de España con datos reales de Airbnb...
